In [5]:
#Importing required libraries
from sklearn import tree
import pandas as pd
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import os

In [6]:
df = pd.read_csv(os.path.join("Resources/healthcare-dataset-stroke-data_nochild.csv"))

In [7]:
mean_value=df['bmi'].mean()
df['bmi_ave']=df['bmi'].fillna(mean_value)

In [8]:
target = df["stroke"]
target_names = ["negative", "positive"]
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_ave
0,Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1,36.600000
1,Female,61,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1,30.421719
2,Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1,32.500000
3,Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1,34.400000
4,Female,79,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1,24.000000


In [9]:
encoded_df = pd.get_dummies(df, columns=['gender','ever_married', 'Residence_type'])
encoded_df

,age,hypertension,heart_disease,work_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_ave,gender_Female,gender_Male,ever_married_No,ever_married_Yes,Residence_type_Rural,Residence_type_Urban
0,67,0,1,Private,228.69,36.6,formerly smoked,1,36.600000,0,1,0,1,0,1
1,61,0,0,Self-employed,202.21,NaN,never smoked,1,30.421719,1,0,0,1,1,0
2,80,0,1,Private,105.92,32.5,never smoked,1,32.500000,0,1,0,1,1,0
3,49,0,0,Private,171.23,34.4,smokes,1,34.400000,1,0,0,1,0,1
4,79,1,0,Self-employed,174.12,24.0,never smoked,1,24.000000,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4308,80,1,0,Private,83.75,NaN,never smoked,0,30.421719,1,0,0,1,0,1
4309,81,0,0,Self-employed,125.20,40.0,never smoked,0,40.000000,1,0,0,1,0,1
4310,35,0,0,Self-employed,82.99,30.6,never smoked,0,30.600000,1,0,0,1,1,0
4311,51,0,0,Private,166.29,25.6,formerly smoked,0,25.600000,0,1,0,1,1,0


In [10]:
encoded_stroke_df = encoded_df.drop(columns=['gender_Female','ever_married_No','Residence_type_Rural'])
encoded_stroke_df

,age,hypertension,heart_disease,work_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_ave,gender_Male,ever_married_Yes,Residence_type_Urban
0,67,0,1,Private,228.69,36.6,formerly smoked,1,36.600000,1,1,1
1,61,0,0,Self-employed,202.21,NaN,never smoked,1,30.421719,0,1,0
2,80,0,1,Private,105.92,32.5,never smoked,1,32.500000,1,1,0
3,49,0,0,Private,171.23,34.4,smokes,1,34.400000,0,1,1
4,79,1,0,Self-employed,174.12,24.0,never smoked,1,24.000000,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4308,80,1,0,Private,83.75,NaN,never smoked,0,30.421719,0,1,1
4309,81,0,0,Self-employed,125.20,40.0,never smoked,0,40.000000,0,1,1
4310,35,0,0,Self-employed,82.99,30.6,never smoked,0,30.600000,0,1,0
4311,51,0,0,Private,166.29,25.6,formerly smoked,0,25.600000,1,1,0


In [11]:
encoded_stroke_df.work_type[encoded_stroke_df.work_type == 'Private'] = 1
encoded_stroke_df.work_type[encoded_stroke_df.work_type == 'Self-employed'] = 2
encoded_stroke_df.work_type[encoded_stroke_df.work_type == 'Govt_job'] = 3
encoded_stroke_df.work_type[encoded_stroke_df.work_type == 'Never_worked'] = 4

#data.Residence_type[data.Residence_type == 'Urban'] = 1
#data.Residence_type[data.Residence_type == 'Rural'] = 2

encoded_stroke_df.smoking_status[encoded_stroke_df.smoking_status == 'formerly smoked'] = 1
encoded_stroke_df.smoking_status[encoded_stroke_df.smoking_status == 'never smoked'] = 2
encoded_stroke_df.smoking_status[encoded_stroke_df.smoking_status == 'smokes'] = 3
encoded_stroke_df.smoking_status[encoded_stroke_df.smoking_status == 'Unknown'] = 4

encoded_stroke_df.head(10)

<ipython-input-11-c27e10f7015c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encoded_stroke_df.work_type[encoded_stroke_df.work_type == 'Private'] = 1
<ipython-input-11-c27e10f7015c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encoded_stroke_df.work_type[encoded_stroke_df.work_type == 'Self-employed'] = 2
<ipython-input-11-c27e10f7015c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encoded_stroke_df.work_type[encoded_stroke

,age,hypertension,heart_disease,work_type,avg_glucose_level,bmi,smoking_status,stroke,bmi_ave,gender_Male,ever_married_Yes,Residence_type_Urban
0,67,0,1,1,228.69,36.6,1,1,36.600000,1,1,1
1,61,0,0,2,202.21,NaN,2,1,30.421719,0,1,0
2,80,0,1,1,105.92,32.5,2,1,32.500000,1,1,0
3,49,0,0,1,171.23,34.4,3,1,34.400000,0,1,1
4,79,1,0,2,174.12,24.0,2,1,24.000000,0,1,0
5,81,0,0,1,186.21,29.0,1,1,29.000000,1,1,1
6,74,1,1,1,70.09,27.4,2,1,27.400000,1,1,0
7,69,0,0,1,94.39,22.8,2,1,22.800000,0,0,1
8,59,0,0,1,76.15,NaN,4,1,30.421719,0,1,0
9,78,0,0,1,58.57,24.2,4,1,24.200000,0,1,1


In [12]:
data = encoded_stroke_df.drop("stroke", axis=1)
feature_names = data.columns
data.head(10)

,age,hypertension,heart_disease,work_type,avg_glucose_level,bmi,smoking_status,bmi_ave,gender_Male,ever_married_Yes,Residence_type_Urban
0,67,0,1,1,228.69,36.6,1,36.600000,1,1,1
1,61,0,0,2,202.21,NaN,2,30.421719,0,1,0
2,80,0,1,1,105.92,32.5,2,32.500000,1,1,0
3,49,0,0,1,171.23,34.4,3,34.400000,0,1,1
4,79,1,0,2,174.12,24.0,2,24.000000,0,1,0
5,81,0,0,1,186.21,29.0,1,29.000000,1,1,1
6,74,1,1,1,70.09,27.4,2,27.400000,1,1,0
7,69,0,0,1,94.39,22.8,2,22.800000,0,0,1
8,59,0,0,1,76.15,NaN,4,30.421719,0,1,0
9,78,0,0,1,58.57,24.2,4,24.200000,0,1,1


In [13]:
data = data.drop(["bmi"],axis=1)
feature_names = data.columns
data.head(10)

#,'work_type','Residence_type_Urban','gender_Male','ever_married_Yes'

,age,hypertension,heart_disease,work_type,avg_glucose_level,smoking_status,bmi_ave,gender_Male,ever_married_Yes,Residence_type_Urban
0,67,0,1,1,228.69,1,36.600000,1,1,1
1,61,0,0,2,202.21,2,30.421719,0,1,0
2,80,0,1,1,105.92,2,32.500000,1,1,0
3,49,0,0,1,171.23,3,34.400000,0,1,1
4,79,1,0,2,174.12,2,24.000000,0,1,0
5,81,0,0,1,186.21,1,29.000000,1,1,1
6,74,1,1,1,70.09,2,27.400000,1,1,0
7,69,0,0,1,94.39,2,22.800000,0,0,1
8,59,0,0,1,76.15,4,30.421719,0,1,0
9,78,0,0,1,58.57,4,24.200000,0,1,1


In [14]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [15]:
#Implementing cross validation
from sklearn.ensemble import RandomForestClassifier 
k = 5
kf = KFold(n_splits=k, random_state=None)

#model = LogisticRegression(solver= 'liblinear') 

model = RandomForestClassifier(n_estimators=200)

acc_score = []

for train_index , test_index in kf.split(X):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = y[train_index] , y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.49130938586326767, 0.4831981460023175, 0.5028968713789108, 0.511600928074246, 0.5232018561484919]
Avg accuracy : 0.5024414374934467
